In [1]:
import sys
sys.path.append('/home/rd804/particle_transformer')
import networks.example_ParticleTransformer as part
import networks.example_ParticleTransformer_finetune as part_finetune
from weaver.utils.dataset import DataConfig
import torch
from weaver.nn.model.ParticleTransformer import ParticleTransformer
from weaver.utils.logger import _logger

In [2]:
cd ..

/home/rd804/particle_transformer


In [3]:
import torch
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [4]:
import h5py
import numpy as np 

In [5]:
data_config_file = 'data/JetClass/JetClass_full.yaml'

In [6]:
data_config = DataConfig.load(data_config_file, load_observers=False, load_reweight_info=False)

In [7]:
# count number of parameters

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)



In [9]:
model, model_info = part_finetune.get_model(data_config)
model.to(device)
model.mod.requires_grad_(False)
print(f'The model has {count_parameters(model):,} trainable parameters')


input_dim: 10
The model has 1,290 trainable parameters


In [11]:
# model

The model has 2,143,486 trainable parameters


In [12]:
qcd = h5py.File("data/JetClass/qcd_N30_100k.hdf5",'r')['4_momenta']

In [13]:
qcd = np.array(qcd)

In [14]:
qcd.shape

(100000, 30, 14)

In [17]:
qcd = qcd[:5]

In [23]:
vector = qcd[:,:,:4]
features = qcd[:,:,4:]
mask = torch.tensor((vector!=[0,0,0,0])[...,0]).unsqueeze(-1)

vector_ = torch.from_numpy(vector).float().to(device)
features_ = torch.from_numpy(features).float().to(device)
mask_ = mask.to(torch.bool).to(device)


vector = torch.swapaxes(vector_, 1, 2)
features = torch.swapaxes(features_, 1, 2)
mask = torch.swapaxes(mask_, 1, 2)
#vector = vector_.reshape(-1, 4, 30)
#features = features_.reshape(-1, 10, 30)
#mask = mask_.reshape(-1, 1, 30)

print(vector.shape, features.shape, mask.shape)

torch.Size([5, 4, 30]) torch.Size([5, 10, 30]) torch.Size([5, 1, 30])


In [28]:
vector_[0][:,0] == vector[0][0,:]

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True], device='cuda:0')

In [25]:
vector[0]

tensor([[ 31.2660,  29.5754,  24.3738,  19.7528,  19.4773,  14.8039,  14.3368,
          14.3163,   9.8281,  10.3337,   9.7158,   9.4207,   9.1491,   8.8331,
           8.6541,   7.9766,   6.1533,   6.2985,   6.5550,   5.7657,   4.9204,
           4.4619,   2.9796,   3.8783,   3.8842,   3.1165,   2.1151,   2.6623,
           2.4259,   2.6129],
        [-38.0751, -39.2075, -28.7359, -25.5304, -24.0376, -19.0401, -18.4265,
         -17.9181, -13.0966, -11.8702, -12.3258, -11.9550, -11.4691, -11.4601,
         -11.4354, -10.0531,  -8.7314,  -7.7697,  -7.5443,  -7.8448,  -7.3548,
          -5.8255,  -5.7022,  -5.0602,  -4.9233,  -4.2517,  -4.4931,  -3.5276,
          -3.6841,  -3.3233],
        [-23.3272, -16.7902, -16.9258, -12.7473, -14.2079, -11.2024,  -8.9283,
         -10.9013,  -5.2647,  -7.3408,  -7.5359,  -6.9933,  -6.2041,  -6.0538,
          -5.4385,  -5.6935,  -3.5830,  -3.5607,  -3.5630,  -3.8287,  -4.0426,
          -4.8051,  -2.1574,  -2.2782,  -2.3322,  -2.0455,  -3.0968,  -

In [21]:
model.forward

<bound method ParticleTransformerWrapper.forward of ParticleTransformerWrapper(
  (mod): ParticleTransformer(
    (trimmer): SequenceTrimmer()
    (embed): Embed(
      (input_bn): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (embed): Sequential(
        (0): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
        (1): Linear(in_features=10, out_features=128, bias=True)
        (2): GELU(approximate='none')
        (3): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (4): Linear(in_features=128, out_features=512, bias=True)
        (5): GELU(approximate='none')
        (6): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (7): Linear(in_features=512, out_features=128, bias=True)
        (8): GELU(approximate='none')
      )
    )
    (pair_embed): PairEmbed(
      (embed): Sequential(
        (0): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv1d(4, 64, kernel_size=

In [29]:
output = model(features, lorentz_vectors=vector , mask=mask)

/home/rd804/.conda/envs/part/lib/python3.10/site-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


In [13]:
path_part = 'models/ParT_full.pt'
model.load_state_dict(torch.load(path_part))

<All keys matched successfully>